In [10]:
# imports

import pickle
import math
import pandas as pd
import numpy as np
import os
import pathlib
import tqdm
from functools import reduce
from scipy.stats import uniform, randint
import sklearn
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
import warnings
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
warnings.filterwarnings("ignore", message="pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.")

# import all_Data object ... if not loaded run clean.ipynb script in code/clean folder to create

proj_dir = '/Users/nickbachelder/Desktop/Kaggle/Linemen'
os.chdir( os.path.join(proj_dir, 'code/clean') )
%run clean_objects.ipynb
os.chdir( os.path.join(proj_dir, 'data') )
with open('all_data.pkl', 'rb') as file:
    all_data = pickle.load(file)
os.chdir( os.path.join(proj_dir, 'code/modeling') )
%run lstm_objects.ipynb
%run metric_objects.ipynb

In [11]:
# get labels and split data

week = 9

all_dat_labels = all_data.get_rush_sequences_labels(week = week, normalize = True)
training_dat, test_dat = train_test_split(all_dat_labels, test_size = 0.2)

# Check for NAs (None)
bad_indexs = []
print('Train NA')
for value in range(len(training_dat)):
    any_na = training_dat[value][0].isnull().values.any()
    if any_na == True:
        bad_indexs.append(value)
        print(value)

for index in sorted(bad_indexs, reverse=True):
    del training_dat[index]


bad_indexs = []
print('Test NA')
for value in range(len(test_dat)):
    any_na = test_dat[value][0].isnull().values.any()
    if any_na == True:
        bad_indexs.append(value)
        print(value)

for index in sorted(bad_indexs, reverse=True):
    del test_dat[index]

print(f'Length train : {len(training_dat)}')
print(f'Length test : {len(test_dat)}')

Train NA
Test NA
Length train : 6816
Length test : 1704


In [12]:
# model building & training

model = SequenceModel(n_features = training_dat[0][0].shape[1], n_classes = 2, n_hidden = 15, n_layers = 1)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= 0.01)
train_dataset = RushDataset(sequences = training_dat, sequence_length=50)
train_loader = DataLoader(train_dataset, batch_size = 50, shuffle = True)
test_dataset = RushDataset(sequences = test_dat, sequence_length=30)
test_loader = DataLoader(test_dataset, batch_size = 50, shuffle = True)

lstm_model = train_rush_lstm(train_loader, test_loader, model, loss_function, optimizer, num_epochs = 20)

Epoch 0
---------
Overall Train loss: 0.6262316521066819 , Overall Train AUC: 0.6403133629651642
Train loss for period 1: 0.672293063062821 , Train AUC for period 1: 0.5294056254622809
Train loss for period 2: 0.6759300110113882 , Train AUC for period 2: 0.5362513102065654
Train loss for period 3: 0.6707644814992473 , Train AUC for period 3: 0.545725984896112
Train loss for period 4: 0.6596401964660978 , Train AUC for period 4: 0.5705685982383231
Train loss for period 5: 0.6382846101357119 , Train AUC for period 5: 0.6204173786881013
Train loss for period 6: 0.6151011714535038 , Train AUC for period 6: 0.6727594186071573
Train loss for period 7: 0.595381057610477 , Train AUC for period 7: 0.7168660821923336
Train loss for period 8: 0.5822511052128172 , Train AUC for period 8: 0.7473951052911297
Train loss for period 9: 0.5771308456459184 , Train AUC for period 9: 0.7590270671320084
Train loss for period 10: 0.5755399728778505 , Train AUC for period 10: 0.7621600880058952
Overall test l

In [13]:
os.chdir( os.path.join(proj_dir, 'data') )
torch.save(lstm_model.state_dict(), "lstm_weights")